In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
use_wandb = True

In [4]:
if use_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: askmuhsin (use `wandb login --relogin` to force relogin)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary

In [6]:
from datasets.cifar10 import trainloader, testloader
from utils.general_utils import setup_env
from utils.training import train_model
from utils.testing import test_model

Files already downloaded and verified
Total number of images:  50000
Mean =  (0.4890062, 0.47970363, 0.47680542) 
 STD =  (0.264582, 0.258996, 0.25643882)
Files already downloaded and verified
Files already downloaded and verified
Batch Size -- 512


In [7]:
from models.model_v7 import Net

In [12]:
cuda, device = setup_env()

train_dataloader, test_dataloader = trainloader, testloader

model = Net(
    batch_norm=True,
    dropout_value=0.0,
).to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
              ReLU-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
           Dropout-8           [-1, 64, 32, 32]               0
        conv_block-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 32, 16, 16]          18,464
           Conv2d-11           [-1, 32, 16, 16]           1,056
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
 transition_block-14           [-1, 32,

In [15]:
if use_wandb:

    wandb.init(
        project='s7_cifar10',              ## unique to each project / assignment
        entity='weights_heist_eva7',       ## this will not change
        ## below are optional but recommended 
        tags=['muhsin'],                   ## can help later to filter runs
        name='version 8 no dropout',    ## a name for the run. 
        notes="dropout - 0.0"
    )

wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = StepLR(optimizer, step_size=12, gamma=0.6)

logs = []
EPOCHS = 100
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_batch_loss, train_batch_acc = train_model(
        model, device, train_dataloader, optimizer, epoch
    )
    test_loss, test_acc = test_model(model, device, test_dataloader)
    scheduler.step()
    
    temp_log = {
        'train_losses': np.mean(train_batch_loss),
        'test_losses': test_loss,
        'train_acc': np.mean(train_batch_acc),
        'test_acc': test_acc,
        "lr": optimizer.param_groups[0]['lr'],
    }
    logs.append(temp_log)
    
    if use_wandb:
        wandb.log(temp_log)

EPOCH: 0


Loss=1.498618721961975 Batch_id=97 Accuracy=33.72: 100%|██████████| 98/98 [00:09<00:00, 10.23it/s] 



Test set: Average loss: 1.6186, Accuracy: 4151/10000 (41.51%)

EPOCH: 1


Loss=1.3299295902252197 Batch_id=97 Accuracy=45.44: 100%|██████████| 98/98 [00:09<00:00,  9.85it/s]



Test set: Average loss: 1.3204, Accuracy: 5213/10000 (52.13%)

EPOCH: 2


...


Test set: Average loss: 0.4374, Accuracy: 8561/10000 (85.61%)

EPOCH: 98


Loss=0.41966643929481506 Batch_id=97 Accuracy=85.01: 100%|██████████| 98/98 [00:09<00:00,  9.86it/s]



Test set: Average loss: 0.4357, Accuracy: 8575/10000 (85.75%)

EPOCH: 99


Loss=0.37477508187294006 Batch_id=97 Accuracy=85.06: 100%|██████████| 98/98 [00:09<00:00, 10.67it/s]



Test set: Average loss: 0.4363, Accuracy: 8563/10000 (85.63%)



In [17]:
if use_wandb:
    wandb.finish()

train_losses,0.43141
test_losses,0.4363
train_acc,85.2107
test_acc,85.63
lr,0.00168
_step,99
_runtime,1095
_timestamp,1637000280


train_losses,█▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_losses,█▆▄▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇█████████████████████████
test_acc,▁▃▅▅▆▇▇▇▇▇▇▇██▇█████████████████████████
lr,█████▅▅▅▅▅▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
